## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-12-02-23-43 +0000,bbc,What we learned from Nicola Sturgeon's memoir,https://www.bbc.com/news/articles/cn844n379y5o...
1,2025-08-12-02-20-01 +0000,startribune,Shooter kills 3 in a Target parking lot in Aus...,https://www.startribune.com/police-say-2-dead-...
2,2025-08-12-02-15-12 +0000,nyt,Footage Shows Medical Volunteer Execution in S...,https://www.nytimes.com/2025/08/11/world/middl...
3,2025-08-12-02-13-54 +0000,nyt,Hundreds of Indian Lawmakers Detained at a Pro...,https://www.nytimes.com/2025/08/11/world/asia/...
4,2025-08-12-02-02-45 +0000,nyt,Military officials testify at California’s sui...,https://www.nytimes.com/live/2025/08/11/us/tru...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2309/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
53,trump,69
78,he,15
122,china,15
37,national,12
147,new,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
12,2025-08-12-01-47-00 +0000,wsj,President Trump said he would deploy National ...,https://www.wsj.com/politics/policy/trump-dc-n...,161
124,2025-08-11-19-43-00 +0000,wsj,The Trump administration’s decision to charge ...,https://www.wsj.com/politics/national-security...,146
194,2025-08-11-14-54-28 +0000,nypost,"Trump takes control of DC police, deploying Na...",https://nypost.com/2025/08/11/us-news/trump-fe...,143
82,2025-08-11-22-10-03 +0000,startribune,Trump says he's placing Washington police unde...,https://www.startribune.com/trump-is-promising...,139
24,2025-08-12-01-00-00 +0000,wsj,President Trump’s decision to deploy National ...,https://www.wsj.com/politics/policy/trump-nati...,134


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
12,161,2025-08-12-01-47-00 +0000,wsj,President Trump said he would deploy National ...,https://www.wsj.com/politics/policy/trump-dc-n...
124,65,2025-08-11-19-43-00 +0000,wsj,The Trump administration’s decision to charge ...,https://www.wsj.com/politics/national-security...
91,45,2025-08-11-21-33-29 +0000,nyt,Israel-Qatar Tensions Escalate After Israel Ki...,https://www.nytimes.com/2025/08/11/world/middl...
213,44,2025-08-11-13-46-00 +0000,wsj,Colombian Presidential Hopeful Dies Two Months...,https://www.wsj.com/world/americas/colombian-p...
168,44,2025-08-11-17-06-32 +0000,bbc,Harry and Meghan sign new multi-year Netflix deal,https://www.bbc.com/news/articles/cvgnne6lznzo...
177,41,2025-08-11-16-09-47 +0000,nypost,Trump says he’ll tell Putin to end Ukraine inv...,https://nypost.com/2025/08/11/world-news/trump...
190,40,2025-08-11-15-02-46 +0000,nypost,Wild video shows Chinese ships crashing while ...,https://nypost.com/2025/08/11/world-news/chine...
127,38,2025-08-11-19-33-26 +0000,nypost,Trump extends China tariff deadline for 90 day...,https://nypost.com/2025/08/11/world-news/trump...
194,32,2025-08-11-14-54-28 +0000,nypost,"Trump takes control of DC police, deploying Na...",https://nypost.com/2025/08/11/us-news/trump-fe...
26,31,2025-08-12-01-00-00 +0000,wsj,Immediately preceding and following the pandem...,https://www.wsj.com/economy/jobs/low-wage-empl...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
